In [1]:
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import cv2


In [2]:
original_root = './smaller-keras-dataset-300/'
new_root = './tvt-yolo-smaller-dataset-300' # tvt = train val test


In [3]:
# only train and test

# Define the split ratio
train_ratio = 0.8

# Create the train and test directories
for split in ['train', 'validation']:
    os.makedirs(os.path.join(new_root, split), exist_ok=True)

# Iterate over each class directory in the original dataset
for class_dir in os.listdir(original_root):
    # Define the paths to the original class directory and the new class directories for train and test
    original_class_dir = os.path.join(original_root, class_dir)
    train_class_dir = os.path.join(new_root, 'train', class_dir)
    test_class_dir = os.path.join(new_root, 'validation', class_dir)

    # Create the new class directories under train and test
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Get all image filenames in the current class directory
    images = [img for img in os.listdir(
        original_class_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Split the images into train and test sets
    train_imgs, test_imgs = train_test_split(images, train_size=train_ratio)

    # Copy the images to the new train and test directories
    for img in train_imgs:
        shutil.copy(os.path.join(original_class_dir, img),
                    os.path.join(train_class_dir, img))

    for img in test_imgs:
        shutil.copy(os.path.join(original_class_dir, img),
                    os.path.join(test_class_dir, img))

print("Dataset split and copied successfully.")


Dataset split and copied successfully.


In [3]:
# train validation and test with randomization

def split_dataset(source_dir, target_dir, train_ratio=0.7, validation_ratio=0.15, test_ratio=0.15):
    classes = os.listdir(source_dir)  # Get all classes

    for cls in classes:
        cls_dir = os.path.join(source_dir, cls)  # Path to each class directory
        images = os.listdir(cls_dir)  # List of images
        np.random.shuffle(images)  # Shuffle the images to randomize

        # Calculate split indices
        total_images = len(images)
        train_end = int(total_images * train_ratio)
        validation_end = train_end + int(total_images * validation_ratio)

        # Split images
        train_images = images[:train_end]
        validation_images = images[train_end:validation_end]
        test_images = images[validation_end:]

        # Function to copy files
        def copy_files(files, split_type):
            split_dir = os.path.join(target_dir, split_type, cls)
            # Create directory if not exists
            os.makedirs(split_dir, exist_ok=True)
            for img in files:
                shutil.copy2(os.path.join(cls_dir, img),
                             os.path.join(split_dir, img))

        # Copy files to their respective directories
        copy_files(train_images, 'train')
        copy_files(validation_images, 'validation')
        copy_files(test_images, 'test')

    print("Dataset split and copied successfully.")

# Split the dataset
split_dataset(original_root, new_root)

Dataset split and copied successfully.


In [4]:
img = cv2.imread('./smaller-keras-dataset-300/acura/slic_0aa5a789508cd718fc4cf044352375ed54080b5e.png')

print(img.shape)


(300, 300, 3)


In [5]:
# model = YOLO('yolov8s-cls.yaml')  # build a new model from YAML
# # load a pretrained model (recommended for training)
model = YOLO('yolov8s-cls.pt')
# # build from YAML and transfer weights
# model = YOLO('yolov8n-cls.yaml').load('yolov8n-cls.pt')

# Train the model
results = model.train(data='./tvt-yolo-smaller-dataset-300/', epochs=25, imgsz=300)


New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.17 🚀 Python-3.9.19 torch-2.2.1 CPU (Apple M1)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=./tvt-yolo-smaller-dataset-300/, epochs=25, time=None, patience=100, batch=16, imgsz=300, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/train... 952 images, 0 corrupt: 100%|██████████| 952/952 [00:00<00:00, 4539.04it/s]

train: New cache created: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/train.cache



val: Scanning /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/validation... 200 images, 0 corrupt: 100%|██████████| 200/200 [00:00<00:00, 7758.25it/s]

val: New cache created: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/validation.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)


...

      25/25         0G    0.04408          8        320: 100%|██████████| 60/60 [02:18<00:00,  2.30s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:13<00:00,  1.94s/it]

                   all      0.985       0.99



25 epochs completed in 1.017 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train/weights/best.pt, 10.3MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.1.17 🚀 Python-3.9.19 torch-2.2.1 CPU (Apple M1)
YOLOv8s-cls summary (fused): 73 layers, 5094415 parameters, 0 gradients, 12.5 GFLOPs
train: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/train... found 952 images in 15 classes ✅ 
val: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/validation... found 200 images in 15 classes ✅ 
test: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/test... found 222 images in 15 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:12<00:00,  1.84s/it]


                   all       0.99      0.995
Speed: 0.0ms preprocess, 60.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


In [6]:
model = YOLO('yolov8s-cls.pt')
model = YOLO('./runs/classify/train_300_b/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1   # top1 accuracy
metrics.top5   # top5 accuracy


Ultralytics YOLOv8.1.17 🚀 Python-3.9.19 torch-2.2.1 CPU (Apple M1)
YOLOv8s-cls summary (fused): 73 layers, 5094415 parameters, 0 gradients, 12.5 GFLOPs
train: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/train... found 952 images in 15 classes ✅ 
val: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/validation... found 200 images in 15 classes ✅ 
test: /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/test... found 222 images in 15 classes ✅ 


val: Scanning /Users/Vikram/Desktop/ECE420/vehicle_logo_segmentation_and_classification/tvt-yolo-smaller-dataset-300/validation... 200 images, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 13/13 [00:13<00:00,  1.01s/it]


                   all       0.99      0.995
Speed: 0.0ms preprocess, 61.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val


0.9950000047683716